In [1]:
!pip install selenium
!pip install webdriver-manager

In [19]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions

from tqdm import tqdm

In [90]:
addr = "https://www.imdb.com/find?q=deadpool&ref_=nv_sr_sm" 

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(addr)


[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\beomj\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache
 


In [91]:
df = pd.read_excel("영화raw데이터.xlsx")
df.openDt.fillna(0, inplace=True)
years = df.openDt.apply(lambda x:str(x)[:4])

In [92]:
indexes = df[df.kor_revenue.notnull()].index.values
print(indexes)
df.head()

[   2    4    7 ... 5788 5807 5823]


,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,prdtStatNm,genres,directors,actors,...,companys,Domestic,Budget,Distributor,review,MPAA,raters,ratings,kor_revenue,kor_audience
0,20203303,지니어스 독,Think Like a Dog,90.0,2020.0,20200916,개봉예정,"코미디,드라마",길 정거,"메간 폭스,조쉬 더하멜,가브리엘 베이트먼",...,"20168728,(주)키다리이엔티,배급사,20153568,(주)스톰픽쳐스코리아,수입사",-,NaN,NaN,NaN,12,667,5.1,NaN,NaN
1,20175405,뉴 뮤턴트,The New Mutants,98.0,2018.0,20200903,개봉예정,"액션,공포(호러),SF",조쉬 분,"메이지 윌리암스,안야 테일러 조이,앨리스 브라가",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20112692,드라이브,Drive,100.0,2011.0,20111117,개봉,"액션,멜로/로맨스",니콜라스 윈딩 레픈,"라이언 고슬링,캐리 멀리건",...,"20111613,주식회사 풍경소리,배급사,20100285,판씨네마(주),배급사,20...","$35,061,555","$15,000,000",FilmDistrict,NaN,NaN,NaN,NaN,156237980.0,20141.0
3,20205511,쥬라기 썬더,Jurassic Thunder,84.0,2019.0,20200923,개봉예정,"액션,SF",NaN,NaN,...,NaN,NaN,NaN,NaN,"....but then again, did you expect an award wi...",15,142,1.9,NaN,NaN
4,20203043,바다 몬스터2,Sea Monsters2,87.0,2018.0,20200812,개봉,애니메이션,에반 트라멜,"서반석,석승훈",...,"20203701,(주)블루라벨픽쳐스,배급사,20190541,(유)헤이데이웍스,수입사",NaN,NaN,NaN,Loose plot with no content. The story is vauge...,NaN,30,3.7,5000.0,1.0


In [93]:
df2 = df[['movieNmEn']]
df2['평점_en'] = None
df2['리뷰_en'] = None
df2['시청등급_en'] = None
df2.head()

,movieNmEn,평점_en,리뷰_en,시청등급_en
0,Think Like a Dog,None,None,None
1,The New Mutants,None,None,None
2,Drive,None,None,None
3,Jurassic Thunder,None,None,None
4,Sea Monsters2,None,None,None


In [94]:
def find(name, year):
    info = []
    reviews = []
    rates = []
    try:
        driver.find_element_by_xpath("/html/body/div[2]/nav/div[2]/div[1]/form/div[2]/div/input").send_keys(name)
        driver.find_element_by_id("suggestion-search-button").click()      
        time.sleep(1.5)
        #------------------
        temp = driver.find_element_by_class_name("findList")
        table = temp.find_elements_by_class_name('result_text')
        found = False
        for temp in table:
            text = temp.text.lower()
            name_split = name.lower().split()
            cnt = 0
            for i in name_split:
                if i in text:
                    cnt +=1
            #print(text, name_split, year)
            if cnt >= len(name_split)/2 and (year in text or str(int(year)-1) in text or str(int(year)+1) in text):
                temp = temp.find_element_by_tag_name("a")
                temp.click()
                time.sleep(1.5)
                found = True
                break

        if not found:
            reviews.append("")
            rates.append("")
            info.append("")
            driver.get(addr)
            return info, reviews, rates
        try:
            #------------------
            rates.append(driver.find_element_by_class_name("imdbRating").text)
            temp = driver.find_element_by_class_name("title_block")
            temp = temp.find_element_by_class_name("subtext").text
            info.append(temp.split("|")[0].strip())

            driver.find_element_by_link_text("USER REVIEWS").click()
            try:
                for i in range(10):
                    driver.find_element_by_id("load-more-trigger").click()
                    time.sleep(2)
            except:
                pass
            time.sleep(1.5)
            #------------------
            temp = driver.find_element_by_class_name("lister-list")
            temp = temp.find_elements_by_class_name("text")
            total = []
            for i in temp:
                total.append(i.text)
            total = "///".join(total)
            reviews.append(total)
        except:
            reviews.append("")
    except:
        reviews.append("")
        rates.append("")
        info.append("")
        driver.get(addr)
    return info, reviews, rates


In [95]:
start = 0
end = 2000
for i in tqdm([i for i in range(start, end) if i in indexes]):
    info, review, rate = find(df2.loc[i,'movieNmEn'], str(int(years[i])))
    df2.loc[i, '리뷰_en']  = review
    df2.loc[i, '평점_en']  = rate
    df2.loc[i, '시청등급_en'] = info


100%|██████████| 1956/1956 [10:46:54<00:00, 19.84s/it]


In [96]:
pd.set_option("display.max_rows", 30)
df2[start:end]

,movieNmEn,평점_en,리뷰_en,시청등급_en
0,Think Like a Dog,None,None,None
1,The New Mutants,None,None,None
2,Drive,"[7.8/10\n558,449]",[///One reviewer here suggested that instead o...,[18]
3,Jurassic Thunder,None,None,None
4,Sea Monsters2,[],[],[]
...,...,...,...,...
1995,Beyond the Lights,"[6.9/10\n15,079]",[I had never heard of this movie before a coll...,[15]
1996,The Equalizer,"[7.2/10\n325,331]","[Really great action sequences, with some nice...",[18]
1997,UNIVERSAL SQUADRONS,[],[],[]
1998,Lullaby,"[6.2/10\n2,528]",[This is a wonderful spectacular and emotional...,[15]


In [97]:
df2.to_excel("1970_2019_review_en"+str(start)+"_"+str(end)+".xlsx", index=False)